In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q evaluate transformers datasets rouge_score jiwer
!pip install -q git+https://github.com/google-research/bleurt.git
!pip install -q nltk textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from evaluate import evaluator
import evaluate
from transformers import AutoModelForSeq2SeqLM, pipeline, AutoTokenizer, AutoModelForSequenceClassification
import nltk
nltk.download('punkt')
import textstat
import torch
import pprint
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#with open("/content/drive/MyDrive/Colab Notebooks/W266/huggingface_credentials.txt", "r") as f:
#    HF_TOKEN = f.read()

In [ ]:
#def calculate_flesch_kincaid(text: list[str]):
#  """Calculates the flesch kincaid score for a list of text"""
#  read_obj = [Readability(t) for t in text]
#  return [r.flesch_kincaid().score for r in read_obj]

entailment_tokenizer = AutoTokenizer.from_pretrained("khalidalt/DeBERTa-v3-large-mnli")
entailment_model = AutoModelForSequenceClassification.from_pretrained("khalidalt/DeBERTa-v3-large-mnli")

def calculate_entailment(
    premise,
    hypothesis,
    device,
    label_names = ["entailment", "neutral", "contradiction"]
):
  input = entailment_tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
  input.to(device)
  entailment_model.to(device)
  output = entailment_model(input["input_ids"])  # device = "cuda:0" or "cpu"

  prediction = torch.softmax(output["logits"][0], -1)

  return label_names[prediction.argmax(0).tolist()]

def encode_decode(df):
    inputs = tokenizer(df[input_col], return_tensors = "pt", max_length = max_input_len, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length = 128)
    result = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {summary_col: result}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
METRIC_1 = "rouge"
METRIC_2 = "bleurt"

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
dataset = load_dataset("jordanfan/processed_us_congress_117_bills_v3")
dataset_test = dataset['test']
cols_to_keep = ["index", "policy_areas", "cur_text", "cleaned_summary",
                             'extracted_text', 'extracted_text_375', 'extracted_text_750', 'extracted_text_1000',
                             'bertsum_extracted_250', 'bertsum_extracted_375',
                             'bertsum_extracted_375_1000', 'bertsum_extracted_250_1000', 'bertsum_extracted_375_750', 'bertsum_extracted_250_750',
                            'bertsum_extracted_375_500', 'bertsum_extracted_250_500', 'bertsum_extracted_375_375', 'bertsum_extracted_250_375']
cols_to_remove = [col for col in dataset_test.column_names if col not in cols_to_keep]
dataset_test = dataset_test.remove_columns(cols_to_remove)

# BART - Untrained

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_untrained = task_evaluator.compute(
    model_or_pipeline="facebook/bart-large",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="cur_text",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
eval_results_untrained_bleurt = eval_results_untrained.pop("scores")
pprint.pprint(eval_results_untrained)
print(np.mean(eval_results_untrained_bleurt))
print(np.median(eval_results_untrained_bleurt))
print(np.std(eval_results_untrained_bleurt))

{'latency_in_seconds': 1.9149694151273213,
 'rouge1': 0.3122221545502962,
 'rouge2': 0.12258266647812757,
 'rougeL': 0.21855152677627418,
 'rougeLsum': 0.21853739239361372,
 'samples_per_second': 0.5222015516804025,
 'total_time_in_seconds': 721.9434695030001}
-0.9303772430916364
-0.9328369498252869
0.31145533337047554


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
max_input_len = tokenizer.model_max_length
input_col = "cur_text"
summary_col = "summary_untrained"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")
#untrained_summaries = test_df["cur_text"].apply(lambda x: encode_decode(x))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/b1565bd7d3a1271f21c9a646b795c5ddcab13e61', commit_message='Upload dataset', commit_description='', oid='b1565bd7d3a1271f21c9a646b795c5ddcab13e61', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_untrained = pd.Series(dataset_test["summary_untrained"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_untrained))
print(np.median(readability_untrained))
print(np.std(readability_untrained))

28.839071618037135
34.46
29.669841262980622


In [ ]:
pd.DataFrame(dataset_test)[["cleaned_summary", "summary_untrained"]].apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_untrained"], device), axis = 1).value_counts(normalize = True)

neutral          0.864721
entailment       0.111406
contradiction    0.023873
Name: proportion, dtype: float64

# BART Baseline - 512

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_baseline_512",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="cur_text",
    label_column="cleaned_summary"
)


In [ ]:
eval_results_bleurt = eval_results.pop("scores")
pprint.pprint(eval_results)
print(np.mean(eval_results_bleurt))
print(np.median(eval_results_bleurt))
print(np.std(eval_results_bleurt))

{'latency_in_seconds': 1.5040594201326258,
 'rouge1': 0.48112005293293464,
 'rouge2': 0.292150933420251,
 'rougeL': 0.37665862841491055,
 'rougeLsum': 0.37680813526828805,
 'samples_per_second': 0.6648673493975533,
 'total_time_in_seconds': 567.03040139}
-0.2049407635269494
-0.2170928418636322
0.32149523677837466


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_baseline_512")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_baseline_512")
max_input_len = 512
input_col = "cur_text"
summary_col = "summary_baseline_512"
dataset_test = dataset_test.map(encode_decode)
#untrained_summaries = test_df["cur_text"].apply(lambda x: encode_decode(x))

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/ed6f751f33d5846ced9451caf4caffe04c6a8e0f', commit_message='Upload dataset', commit_description='', oid='ed6f751f33d5846ced9451caf4caffe04c6a8e0f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_baseline_512 = pd.Series(dataset_test["summary_baseline_512"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_baseline_512))
print(np.median(readability_baseline_512))
print(np.std(readability_baseline_512))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_baseline_512"], device), axis = 1).value_counts(normalize = True))

26.97209549071618
29.18
19.867804183880633
neutral          0.636605
entailment       0.328912
contradiction    0.034483
Name: proportion, dtype: float64


# BART Baseline - PEFT

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_peft = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_baseline_peft",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="extracted_text",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 128, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 128, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 128, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your 

In [ ]:
eval_results_peft_bleurt = eval_results_peft.pop("scores")
pprint.pprint(eval_results_peft)
print(np.mean(eval_results_peft_bleurt))
print(np.median(eval_results_peft_bleurt))
print(np.std(eval_results_peft_bleurt))

{'latency_in_seconds': 1.1907940041352785,
 'rouge1': 0.4533476874027371,
 'rouge2': 0.2628743474559103,
 'rougeL': 0.3557031285918235,
 'rougeLsum': 0.35566309583153655,
 'samples_per_second': 0.8397758105325465,
 'total_time_in_seconds': 448.929339559}
-0.2850662555832129
-0.2944364845752716
0.3562360852454668


# BART Baseline - 1024

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_1024 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_baseline_1024",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="extracted_text",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 128, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 128, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 128, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your 

In [ ]:
eval_results_1024_bleurt = eval_results_1024.pop("scores")
pprint.pprint(eval_results_1024)
print(np.mean(eval_results_1024_bleurt))
print(np.median(eval_results_1024_bleurt))
print(np.std(eval_results_1024_bleurt))

{'latency_in_seconds': 1.2012484259416443,
 'rouge1': 0.47381265862694,
 'rouge2': 0.2808336762975292,
 'rougeL': 0.36981601262365427,
 'rougeLsum': 0.3698649926029271,
 'samples_per_second': 0.832467271885174,
 'total_time_in_seconds': 452.87065657999983}
-0.24579534085778723
-0.26606911420822144
0.34359058027539957


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_baseline_1024")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_baseline_1024")
max_input_len = 1024
input_col = "cur_text"
summary_col = "summary_baseline_1024"
dataset_test = dataset_test.map(encode_decode)
#untrained_summaries = test_df["cur_text"].apply(lambda x: encode_decode(x))

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/5c4706ee02d73c09cd91d931598d96481bc3e10d', commit_message='Upload dataset', commit_description='', oid='5c4706ee02d73c09cd91d931598d96481bc3e10d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_baseline_1024 = pd.Series(dataset_test["summary_baseline_1024"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_baseline_1024))
print(np.median(readability_baseline_1024))
print(np.std(readability_baseline_1024))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_baseline_1024"], device), axis = 1).value_counts(normalize = True))

24.471193633952254
25.46
22.286277097153306
neutral          0.572944
entailment       0.389920
contradiction    0.037135
Name: proportion, dtype: float64


# BART Simple Extraction - 512 input tokens / 375 words

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_extractive0 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_extractive_512_375",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="extracted_text_375",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 128, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 128, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 128, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your 

In [ ]:
eval_results_extractive_bleurt0 = eval_results_extractive0.pop("scores")
pprint.pprint(eval_results_extractive0)
print(np.mean(eval_results_extractive_bleurt0))
print(np.median(eval_results_extractive_bleurt0))
print(np.std(eval_results_extractive_bleurt0))

{'latency_in_seconds': 1.1760398483448276,
 'rouge1': 0.4836414332105021,
 'rouge2': 0.2891725630595779,
 'rougeL': 0.3749263305545452,
 'rougeLsum': 0.3752079272571339,
 'samples_per_second': 0.850311323555415,
 'total_time_in_seconds': 443.36702282600004}
-0.2196366719763222
-0.23987436294555664
0.33188283898534787


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_extractive_512_375")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_extractive_512_375")
max_input_len = 512
input_col = "extracted_text_375"
summary_col = "summary_extractive_512_375"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/25b1e5902ce5108fcb4b68f036de72378c184004', commit_message='Upload dataset', commit_description='', oid='25b1e5902ce5108fcb4b68f036de72378c184004', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_extractive_512_375 = pd.Series(dataset_test["summary_extractive_512_375"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_512_375))
print(np.median(readability_extractive_512_375))
print(np.std(readability_extractive_512_375))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_extractive_512_375"], device), axis = 1).value_counts(normalize = True))

26.717453580901857
29.18
20.376099703797387
neutral          0.612732
entailment       0.347480
contradiction    0.039788
Name: proportion, dtype: float64


# BART Simple Extraction - 512 input tokens / 500 words

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_extractive1 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_extractive_512_500",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="extracted_text",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 128, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 128, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 128, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your 

In [ ]:
eval_results_extractive_bleurt1 = eval_results_extractive1.pop("scores")
pprint.pprint(eval_results_extractive1)
print(np.mean(eval_results_extractive_bleurt1))
print(np.median(eval_results_extractive_bleurt1))
print(np.std(eval_results_extractive_bleurt1))

{'latency_in_seconds': 1.2229472039575606,
 'rouge1': 0.4809552930088512,
 'rouge2': 0.2922391565002364,
 'rougeL': 0.37658098409841095,
 'rougeLsum': 0.37671538739522054,
 'samples_per_second': 0.817696787534176,
 'total_time_in_seconds': 461.0510958920004}
-0.21621547951622413
-0.22538699209690094
0.34377802644300465


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_extractive_512_500")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_extractive_512_500")
max_input_len = 512
input_col = "extracted_text"
summary_col = "summary_extractive_512_500"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/a873d577362f1a267702c7400cfa9e4bf656da1b', commit_message='Upload dataset', commit_description='', oid='a873d577362f1a267702c7400cfa9e4bf656da1b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_extractive_512_500 = pd.Series(dataset_test["summary_extractive_512_500"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_512_500))
print(np.median(readability_extractive_512_500))
print(np.std(readability_extractive_512_500))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_extractive_512_500"], device), axis = 1).value_counts(normalize = True))

26.583421750663128
28.51
20.553032722061136
neutral          0.631300
entailment       0.318302
contradiction    0.050398
Name: proportion, dtype: float64


# BART Simple Extraction -  1024 Input Tokens / 750 words

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_extractive2 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_extractive_1024_750",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="extracted_text_750",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 128, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 128, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)
Your max_length is set to 128, but your input_length is only 61. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


In [ ]:
eval_results_extractive2_bleurt = eval_results_extractive2.pop("scores")
pprint.pprint(eval_results_extractive2)
print(np.mean(eval_results_extractive2_bleurt))
print(np.median(eval_results_extractive2_bleurt))
print(np.std(eval_results_extractive2_bleurt))

{'latency_in_seconds': 1.3141035994323602,
 'rouge1': 0.48522026649949673,
 'rouge2': 0.2963857826128894,
 'rougeL': 0.38319623396297164,
 'rougeLsum': 0.3834727625376466,
 'samples_per_second': 0.7609750102137759,
 'total_time_in_seconds': 495.4170569859998}
-0.21666119733840147
-0.22511547803878784
0.35890971003461114


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_extractive_1024_750")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_extractive_1024_750")
max_input_len = 1024
input_col = "extracted_text_750"
summary_col = "summary_extractive_1024_750"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/738f67a2adae250b7bf8fc4bf4a61fe952436687', commit_message='Upload dataset', commit_description='', oid='738f67a2adae250b7bf8fc4bf4a61fe952436687', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_extractive_1024_750 = pd.Series(dataset_test["summary_extractive_1024_750"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_1024_750))
print(np.median(readability_extractive_1024_750))
print(np.std(readability_extractive_1024_750))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_extractive_1024_750"], device), axis = 1).value_counts(normalize = True))

24.592732095490717
25.12
21.416045342358284
neutral          0.586207
entailment       0.389920
contradiction    0.023873
Name: proportion, dtype: float64


# BART Simple Extraction - 1024 Input Tokens / 1000 words

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_extractive3 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_extractive_1024_1000",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="extracted_text_1000",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)
Your max_length is set to 128, but your input_length is only 2. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 128, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)


In [ ]:
eval_results_extractive3_bleurt = eval_results_extractive3.pop("scores")
pprint.pprint(eval_results_extractive3)
print(np.mean(eval_results_extractive3_bleurt))
print(np.median(eval_results_extractive3_bleurt))
print(np.std(eval_results_extractive3_bleurt))

{'latency_in_seconds': 1.3398017341697614,
 'rouge1': 0.4916821686824279,
 'rouge2': 0.3032846243343599,
 'rougeL': 0.39120013178079294,
 'rougeLsum': 0.3909221688151678,
 'samples_per_second': 0.7463790906493136,
 'total_time_in_seconds': 505.1052537820001}
-0.20906838821123702
-0.216058611869812
0.35246669871579206


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_extractive_1024_1000")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_extractive_1024_1000")
max_input_len = 1024
input_col = "extracted_text_1000"
summary_col = "summary_extractive_1024_1000"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/29af2d36d36dcdb64985be7a96bb9762241b6edb', commit_message='Upload dataset', commit_description='', oid='29af2d36d36dcdb64985be7a96bb9762241b6edb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_extractive_1024_1000 = pd.Series(dataset_test["summary_extractive_1024_1000"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_extractive_1024_1000))
print(np.median(readability_extractive_1024_1000))
print(np.std(readability_extractive_1024_1000))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_extractive_1024_1000"], device), axis = 1).value_counts(normalize = True))

23.624880636604775
24.78
21.75150761076551
neutral          0.644562
entailment       0.339523
contradiction    0.015915
Name: proportion, dtype: float64


# BART BillSum Abstractive - 1024 Input Tokens / 1000 words

In [ ]:
dataset_billsum = load_dataset("jordanfan/billsum_abstracted_us_congress_117_bills_all")
dataset_billsum_test = dataset_billsum['test']
dataset_billsum_test

Generating train split:   0%|          | 0/10988 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/3310 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/367 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'index', 'id', 'policy_areas', 'cur_summary', 'cur_text', 'title', 'titles_official', 'titles_short', 'sponsor_name', 'sponsor_party', 'sponsor_state', 'cleaned_summary', 'extracted_text', 'extracted_text_375', 'extracted_text_750', 'extracted_text_1000', 'bertsum_extracted_250', 'bertsum_extracted_375', 'bertsum_extracted_375_1000', 'bertsum_extracted_250_1000', 'bertsum_extracted_375_750', 'bertsum_extracted_250_750', 'bertsum_extracted_375_500', 'bertsum_extracted_250_500', 'bertsum_extracted_375_375', 'bertsum_extracted_250_375', 'text_len', 'billsum_abstracted_1000', 'billsum_abstracted_500', '__index_level_0__'],
    num_rows: 367
})

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_billsum_abstractive = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_billsum_abstractive_1024_1000",
    data=dataset_billsum_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="billsum_abstracted_1000",
    label_column="cleaned_summary"
)


KeyboardInterrupt: 

In [ ]:
eval_results_billsum_abstractive_bleurt = eval_results_billsum_abstractive.pop("scores")
pprint.pprint(eval_results_billsum_abstractive)
print(np.mean(eval_results_billsum_abstractive_bleurt))
print(np.median(eval_results_billsum_abstractive_bleurt))
print(np.std(eval_results_billsum_abstractive_bleurt))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_billsum_abstractive_1024_1000")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_billsum_abstractive_1024_1000")
max_input_len = 1024
input_col = "billsum_abstracted_1000"
summary_col = "summary_billsum_abstracted_1000"
dataset_billsum_test = dataset_billsum_test.map(encode_decode)
dataset_billsum_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries_billsum")

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries_billsum/commit/832f297a441200e927c0315f400cc6497f55fc9a', commit_message='Upload dataset', commit_description='', oid='832f297a441200e927c0315f400cc6497f55fc9a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_abstractive_1000 = pd.Series(dataset_billsum_test["summary_billsum_abstracted_1000"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_abstractive_1000))
print(np.median(readability_abstractive_1000))
print(np.std(readability_abstractive_1000))
print(pd.DataFrame(dataset_billsum_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_billsum_abstracted_1000"], device), axis = 1).value_counts(normalize = True))

27.167711171662123
27.66
20.103654749094794
neutral          0.623978
entailment       0.318801
contradiction    0.057221
Name: proportion, dtype: float64


# BART BertSum Extractive - 1024 Input Tokens / 375 word chunks / 1000 words

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_bertsum0 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_bertsum_1024_375_1000",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="bertsum_extracted_375_1000",
    label_column="cleaned_summary"
)


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 128, but your input_length is only 72. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 128, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 128, but your input_length is only 64. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
You

In [ ]:
eval_results_bertsum_bleurt0 = eval_results_bertsum0.pop("scores")
pprint.pprint(eval_results_bertsum0)
print(np.mean(eval_results_bertsum_bleurt0))
print(np.median(eval_results_bertsum_bleurt0))
print(np.std(eval_results_bertsum_bleurt0))

{'latency_in_seconds': 1.1323966094562337,
 'rouge1': 0.4596722309679159,
 'rouge2': 0.2735492609192207,
 'rougeL': 0.3605402312321862,
 'rougeLsum': 0.3595810513902834,
 'samples_per_second': 0.8830828277384112,
 'total_time_in_seconds': 426.91352176500004}
-0.2361670017084329
-0.2376480996608734
0.31787109018687665


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_bertsum_1024_375_1000")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_bertsum_1024_375_1000")
max_input_len = 1024
input_col = "bertsum_extracted_375_1000"
summary_col = "summary_bertsum_1024_375_1000"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/8152af8df5427e433acbea38da68e70004882cc0', commit_message='Upload dataset', commit_description='', oid='8152af8df5427e433acbea38da68e70004882cc0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_bertsum_375_1000 = pd.Series(dataset_test["summary_bertsum_1024_375_1000"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_bertsum_375_1000))
print(np.median(readability_bertsum_375_1000))
print(np.std(readability_bertsum_375_1000))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_bertsum_1024_375_1000"], device), axis = 1).value_counts(normalize = True))

29.20397877984085
30.54
18.895446101469926
neutral          0.625995
entailment       0.336870
contradiction    0.037135
Name: proportion, dtype: float64


# BART BertSum Extractive - 1024 Input Tokens / 250 word chunks / 1000 words

In [ ]:
task_evaluator = evaluator("summarization")

# 1. Pass a model name or path
eval_results_bertsum1 = task_evaluator.compute(
    model_or_pipeline="jordanfan/bart_bertsum_1024_250_1000",
    data=dataset_test,
    metric = evaluate.combine([METRIC_1, METRIC_2]),
    input_column="bertsum_extracted_250_1000",
    label_column="cleaned_summary"
)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Your max_length is set to 128, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 128, but your input_length is only 111. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Your max_length is set to 128, but your input_length is only 111. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Your max_length is set to 128, but your input_length is only 64. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Yo

In [ ]:
eval_results_bertsum_bleurt1 = eval_results_bertsum1.pop("scores")
pprint.pprint(eval_results_bertsum1)
print(np.mean(eval_results_bertsum_bleurt1))
print(np.median(eval_results_bertsum_bleurt1))
print(np.std(eval_results_bertsum_bleurt1))

{'latency_in_seconds': 1.1327221126047762,
 'rouge1': 0.4648985939349028,
 'rouge2': 0.27671195944505966,
 'rougeL': 0.3690973406996343,
 'rougeLsum': 0.3692498235625655,
 'samples_per_second': 0.8828290618432688,
 'total_time_in_seconds': 427.0362364520006}
-0.22711420156041887
-0.2431623786687851
0.3260156504422369


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jordanfan/bart_bertsum_1024_250_1000")
model = AutoModelForSeq2SeqLM.from_pretrained("jordanfan/bart_bertsum_1024_250_1000")
max_input_len = 1024
input_col = "bertsum_extracted_250_1000"
summary_col = "summary_bertsum_1024_250_1000"
dataset_test = dataset_test.map(encode_decode)
dataset_test.push_to_hub("jordanfan/congress_117_bills_test_bart_summaries")

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/congress_117_bills_test_bart_summaries/commit/a47eb423f76cfc16b787e562515db3b4d3c8f339', commit_message='Upload dataset', commit_description='', oid='a47eb423f76cfc16b787e562515db3b4d3c8f339', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
readability_bertsum_250_1000 = pd.Series(dataset_test["summary_bertsum_1024_250_1000"]).map(lambda x: textstat.flesch_reading_ease(x))
print(np.mean(readability_bertsum_250_1000))
print(np.median(readability_bertsum_250_1000))
print(np.std(readability_bertsum_250_1000))
print(pd.DataFrame(dataset_test).apply(
    lambda x: calculate_entailment(x["cleaned_summary"], x["summary_bertsum_1024_250_1000"], device), axis = 1).value_counts(normalize = True))

27.86610079575597
28.33
18.831576800581683
neutral          0.610080
entailment       0.339523
contradiction    0.050398
Name: proportion, dtype: float64
